In [0]:
#!pip install pd

In [0]:
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression, LinearRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier,  BaggingClassifier, BaggingRegressor, RandomTreesEmbedding,GradientBoostingClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk import word_tokenize
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import FunctionTransformer
import re
import nltk
import gensim
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
sno = nltk.stem.SnowballStemmer('english')

In [44]:
data_url = 'https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv'
data = pd.read_csv(data_url, sep='\t')
data.head()

,COMPLAINT_ID,DATE,PRODUCT_ID,ISSUE_ID,cleaned_text
0,3178905,03/13/2019,44,318,go year . contact advis never took loan . advi...
1,3175952,03/12/2019,44,349,"mail valid debt xx/xx/19 valid receiv , receiv..."
2,3174747,03/09/2019,44,16,xx/xx/xxxx appli receiv onlin loan bluechip fi...
3,3173291,03/08/2019,44,16,xx/xx/xxxx appli receiv onlin loan . loan amou...
4,3172221,03/07/2019,44,48,told husband left bill . debt would pay within...


Попробуйте поднять качество классификации до 70% с помощью известных вам методов препроцессинга и Voting

In [0]:
stop_words =  set(stopwords.words('english')) 

In [0]:
def stem_tokenizer(text):
    return [sno.stem(t) for t in word_tokenize(text)]

In [0]:
def preprocessing(line): #удаляем стоп-слова
  line_token = stem_tokenizer(line)
  filtered_line = [w for w in line_token if not w in stop_words]
  filtered_line = ' '.join(filtered_line)
  return filtered_line

In [0]:
data["preprocessed_text"] = data["cleaned_text"].apply(preprocessing)

In [0]:
y = data["PRODUCT_ID"]
X = data["preprocessed_text"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [50]:
model = gensim.models.Word2Vec(X_train, size=100)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [0]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = 100

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [51]:
clf1 = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42)
clf2 = RandomForestClassifier(n_estimators=50, random_state=42)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

voting = Pipeline([
    ('feats', FeatureUnion([
        ('tfidf', TfidfVectorizer(tokenizer=stem_tokenizer)),
        ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v))])),
    ('clf', SVC(class_weight="balanced", random_state =42))
    ])

voting = voting.fit(X_train, y_train)
predictions = voting.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))


Precision:   0.71
Recall:   0.71
F1-measure:   0.71
Accuracy:   0.71


Основываясь на своих знаниях об ансамблировании различных моделей, на данных из complaints.csv достигните наилучшего результата

In [0]:
data_url = 'https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv'
data = pd.read_csv(data_url, sep='\t')
data.head()
y = data["PRODUCT_ID"]
X = data["cleaned_text"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [0]:
cv = CountVectorizer(tokenizer=stem_tokenizer, max_features=800, min_df=0.001, max_df=0.6)
X = cv.fit_transform(X)

##Bagging

In [0]:
clf1 = GradientBoostingClassifier(n_estimators=150,  random_state=42)
clf2 = RandomForestClassifier(n_estimators=150, random_state=42)
clf3 = ExtraTreesClassifier(n_estimators=150, random_state=42)

bagging1 = BaggingClassifier(base_estimator=clf1, n_estimators=10, max_samples=0.8, max_features=0.8)
bagging2 = BaggingClassifier(base_estimator=clf2, n_estimators=10, max_samples=0.8, max_features=0.8)
bagging3 = BaggingClassifier(base_estimator=clf2, n_estimators=10, max_samples=0.8, max_features=0.8)

In [84]:
label = ['Gradient Boosting', 'RandomForestClassifier', 'ExtraTreesClassifier', 'Bagging Gradient Boosting', 'Bagging Random Forest Classifier', 'Bagging Extra Trees Classifier']
clf_list = [clf1, clf2, clf3, clf4, clf5, bagging1, bagging2, bagging3, bagging4, bagging5]
for clf, label in zip(clf_list, label):        
    scores = cross_val_score(clf, X, y, cv=3, scoring='accuracy')
    print ("Accuracy: %.2f (+/- %.2f) [%s]" %(scores.mean(), scores.std(), label))

Accuracy: 0.63 (+/- 0.01) [Gradient Boosting]
Accuracy: 0.63 (+/- 0.01) [RandomForestClassifier]
Accuracy: 0.62 (+/- 0.01) [ExtraTreesClassifier]
Accuracy: 0.63 (+/- 0.01) [Bagging Gradient Boosting]
Accuracy: 0.62 (+/- 0.01) [Bagging Random Forest Classifier]
Accuracy: 0.64 (+/- 0.02) [Bagging Extra Trees Classifier]


##Boosting

In [0]:
clf = RandomForestClassifier(n_estimators=150, random_state=42)

num_est = [1, 2, 3, 10]
label = ['AdaBoost (n_est=1)', 'AdaBoost (n_est=2)', 'AdaBoost (n_est=3)', 'AdaBoost (n_est=10)']

In [107]:
for n_est, label in zip(num_est, label):   
  boosting = AdaBoostClassifier(base_estimator=clf, n_estimators=n_est)   
  boosting.fit(X, y) 
  scores = cross_val_score(clf, X, y, cv=3, scoring='accuracy')
  print ("Accuracy: %.2f (+/- %.2f) [%s]" %(scores.mean(), scores.std(), label))

Accuracy: 0.63 (+/- 0.01) [AdaBoost (n_est=1)]
Accuracy: 0.63 (+/- 0.01) [AdaBoost (n_est=2)]
Accuracy: 0.63 (+/- 0.01) [AdaBoost (n_est=3)]
Accuracy: 0.63 (+/- 0.01) [AdaBoost (n_est=10)]


##Stacking

In [0]:
clf1 = GradientBoostingClassifier(n_estimators=150,  random_state=42)
clf2 = RandomForestClassifier(n_estimators=150, random_state=42)
clf3 = ExtraTreesClassifier(n_estimators=150, random_state=42)
lr = LogisticRegression(max_iter=50000)
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3, clf4], 
                          meta_classifier=lr)

In [0]:
label = ['GradientBoosting', 'RandomForestClassifier', 'Extra Trees', 'Stacking Classifier']
clf_list = [clf1, clf2, clf3, sclf]

In [101]:
clf_cv_mean = []
clf_cv_std = []
for clf, label in zip(clf_list, label):
    scores = cross_val_score(clf, X, y, cv=3, scoring='accuracy')
    print ("Accuracy: %.2f (+/- %.2f) [%s]" %(scores.mean(), scores.std(), label))
    clf_cv_mean.append(scores.mean())
    clf_cv_std.append(scores.std())

Accuracy: 0.63 (+/- 0.01) [GradientBoosting]
Accuracy: 0.63 (+/- 0.01) [RandomForestClassifier]
Accuracy: 0.62 (+/- 0.01) [Extra Trees]
Accuracy: 0.60 (+/- 0.01) [Stacking Classifier]
